In [12]:
# Dependencies

import pandas as pd
import numpy as np

import torch
from torch.utils.data import DataLoader, TensorDataset

# Importing ALBERT
from transformers import AlbertTokenizer, AlbertForSequenceClassification
from torch.optim import AdamW

# Processing

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [13]:
df = pd.read_csv(r"../Data/processedData.csv")

X_train, X_test, y_train, y_test = train_test_split(df[['processedMessage','fingers','tail']],
                                                    df['species_group'])

label_encoder = LabelEncoder()
label_encoder.fit(df['species_group'])

y_train = torch.tensor(to_categorical(label_encoder.transform(y_train),num_classes=12))
y_test = torch.tensor(to_categorical(label_encoder.transform(y_test),num_classes=12))

In [16]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=12)

X_train_encodings = tokenizer(list(X_train['processedMessage']), return_tensors='pt', padding=True, truncation=True, max_length=128)
X_test_encodings = tokenizer(list(X_test['processedMessage']), return_tensors='pt', padding=True, truncation=True, max_length=128)

train_dataset = TensorDataset(X_train_encodings['input_ids'], X_train_encodings['attention_mask'], y_train)
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)

test_dataset = TensorDataset(X_test_encodings['input_ids'], X_test_encodings['attention_mask'], y_test)
test_loader = DataLoader(test_dataset, batch_size=12, shuffle=False)

optimizer = AdamW(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss()

C:\Users\suhan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train()

num_epochs = 10

for epoch in range(num_epochs):
    
    correct_predictions = 0

    for batch in train_loader:

        input_ids, attention_mask, labels = batch

        outputs = model(input_ids, attention_mask, labels)
        loss = outputs.loss
        logits = outputs.logits

        optimizer.zero_grad()
        loss_fn.backward()
        optimizer.step()

        predictions = torch.argmax(logits, dim=-1)
